In [5]:
import open3d as o3d
import numpy as np
import os
import sys
import math
import random
from time import sleep
# monkey patches visualization and provides helpers to load geometries
#sys.path.append('..')
#import open3d_tutorial as o3dtut
# change to True if you want to interact with the visualization windows
#o3dtut.interactive = not "CI" in os.environ

In [9]:
class cube:
    def __init__(self, x, y, z, w, l, h, thetaX, thetaY, thetaZ, thresh):
        self.x = x
        self.y = y
        self.z = z
        self.w = w
        self.l = l
        self.h = h
        
        self.thetaX = thetaX
        self.thetaY = thetaY
        self.thetaZ = thetaZ
         
        self.centerx = x + w/2
        self.centery = y + l/2
        self.centerz = z + h/2
        
        ptA = [x, y, z]
        ptB = [x+self.w, y, z]
        ptC = [x, y+self.l, z]
        ptD = [x+self.w, y+self.l, z]
        ptE = [x, y, z+self.h]
        ptF = [x+self.w, y, z+self.h]
        ptG = [x, y+self.l, z+self.h]
        ptH = [x+self.w, y+self.l, z+self.h]
    
        pointsInit = [
            ptA,
            ptB,
            ptC,
            ptD,
            ptE,
            ptF,
            ptG,
            ptH
        ]
        ptA1 = [x-thresh, y-thresh, z-thresh]
        ptB1 = [x+self.w+thresh, y-thresh, z-thresh]
        ptC1 = [x-thresh, y+self.l+thresh, z-thresh]
        ptD1 = [x+self.w+thresh, y+self.l+thresh, z-thresh]
        ptE1 = [x-thresh, y-thresh, z+self.h+thresh]
        ptF1 = [x+self.w+thresh, y-thresh, z+self.h+thresh]
        ptG1 = [x-thresh, y+self.l+thresh, z+self.h+thresh]
        ptH1 = [x+self.w+thresh, y+self.l+thresh, z+self.h+thresh]
    
        pointsInit1 = [
            ptA1,
            ptB1,
            ptC1,
            ptD1,
            ptE1,
            ptF1,
            ptG1,
            ptH1
        ]
        lines = [
            [0, 1],
            [0, 2],
            [1, 3],
            [2, 3],
            [4, 5],
            [4, 6],
            [5, 7],
            [6, 7],
            [0, 4],
            [1, 5],
            [2, 6],
            [3, 7],
        ]
        lines1 = [
            [0, 1],
            [0, 2],
            [1, 3],
            [2, 3],
            [4, 5],
            [4, 6],
            [5, 7],
            [6, 7],
            [0, 4],
            [1, 5],
            [2, 6],
            [3, 7],
        ]
        
        colors = [[1, 0, 0] for i in range(len(lines))]
        colors1 = [[0, 1, 0] for i in range(len(lines1))]
        
        self.line_set = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(pointsInit),
            lines=o3d.utility.Vector2iVector(lines),
        )
        self.line_set1 = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(pointsInit1),
            lines=o3d.utility.Vector2iVector(lines1),
        )
        self.line_set.colors = o3d.utility.Vector3dVector(colors)
        self.line_set1.colors = o3d.utility.Vector3dVector(colors1)
        rotation=self.line_set.get_rotation_matrix_from_xyz((math.radians(self.thetaX),math.radians(self.thetaY),math.radians(self.thetaZ)))
        #rotation1=self.line_set1.get_rotation_matrix_from_xyz((math.radians(self.thetaX),math.radians(self.thetaY),math.radians(self.thetaZ)))
        self.line_set.rotate(rotation, (self.centerx,self.centery,self.centerz))
        self.line_set1.rotate(rotation, (self.centerx,self.centery,self.centerz))
        
    def check_pts(self):
        pts_tfd = np.asarray(self.line_set.points)
        ptA, ptB, ptC, ptD, ptE, ptF, ptG, ptH = pts_tfd
        pts_tfd1 = np.asarray(self.line_set1.points)
        ptA1, ptB1, ptC1, ptD1, ptE1, ptF1, ptG1, ptH1 = pts_tfd1
        
        dir1 = (ptB-ptA)
        size1 = np.linalg.norm(dir1)
        dir1 = dir1 / size1
        
        dir2 = (ptC-ptA)
        size2 = np.linalg.norm(dir2)
        dir2 = dir2 / size2

        dir3 = (ptE-ptA)
        size3 = np.linalg.norm(dir3)
        dir3 = dir3 / size3
        
        dir11 = (ptB1-ptA1)
        size11 = np.linalg.norm(dir11)
        dir11 = dir11 / size11
        
        dir21 = (ptC1-ptA1)
        size21 = np.linalg.norm(dir21)
        dir21 = dir21 / size21

        dir31 = (ptE1-ptA1)
        size31 = np.linalg.norm(dir31)
        dir31 = dir31 / size31

        cube_center = np.array([self.centerx,self.centery,self.centerz])
        dir_vec = np.array([0.3,2,1.1]) - cube_center
        
        #print(cube_center)
        res1 = np.where( (np.absolute(np.dot(dir_vec, dir1)) * 2) > size1 )[0]
        res2 = np.where( (np.absolute(np.dot(dir_vec, dir2)) * 2) > size2 )[0]
        res3 = np.where( (np.absolute(np.dot(dir_vec, dir3)) * 2) > size3 )[0]
        
        res11 = np.where( (np.absolute(np.dot(dir_vec, dir11)) * 2) > size11 )[0]
        res21 = np.where( (np.absolute(np.dot(dir_vec, dir21)) * 2) > size21 )[0]
        res31 = np.where( (np.absolute(np.dot(dir_vec, dir31)) * 2) > size31 )[0]
        #print(res1,res2,res3)
        fits = len(list( set().union(res1, res2, res3) ))
        fits1 = len(list( set().union(res11, res21, res31) ))
        if fits == 0 and fits1 == 1:
            return True
        else:
            return False

In [10]:
class GA:
    def __init__(self):
        self.population=[]
        self.fitness =[]
        
    def init_popultation(self, size):
        #self.population=[]
        self.size = size
        for _ in range(self.size):
            rand_x = random.uniform(-4, 4)
            rand_y = random.uniform(-4, 4)
            rand_z = random.uniform(-4, 4)
            rand_w = random.uniform(0.4, 5)
            rand_l = random.uniform(0.4, 5)
            rand_h = random.uniform(0.4, 5)
            rand_thetaX = random.uniform(0, 360)
            rand_thetaY = random.uniform(0, 360)
            rand_thetaZ = random.uniform(0, 360)
            self.population.append(cube(rand_x,rand_y,rand_z,rand_w,rand_l,rand_h,rand_thetaX,rand_thetaY,rand_thetaZ,0.05))
            
            
 
            
 

In [11]:
if __name__ == '__main__':
    vis = o3d.visualization.Visualizer()
    algorithm = GA()
    algorithm.init_popultation(1)
    vis.create_window(width = 800, height = 600)
    for i in range(len(algorithm.population)):
        vis.add_geometry(algorithm.population[i].line_set)
        vis.add_geometry(algorithm.population[i].line_set1)
        inlier = algorithm.population[i].check_pts()
        print(inlier) #if len = 1 - no intersection
    vis.run()
    vis.destroy_window()

False


In [48]:
vis = o3d.visualization.Visualizer()
print("Let's draw a box using o3d.geometry.LineSet.")
points = [
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [1, 1, 0],
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 1],
]
lines = [
    [0, 1],
    [0, 2],
    [1, 3],
    [2, 3],
    [4, 5],
    [4, 6],
    [5, 7],
    [6, 7],
    [0, 4],
    [1, 5],
    [2, 6],
    [3, 7],
]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)


line_set.colors = o3d.utility.Vector3dVector(colors)
#pcd1 = o3d.PointCloud()
pcd = o3d.io.read_point_cloud("models/cube_test1.ply")
vis.create_window()
geometry1 = line_set
print(np.asarray(line_set.points))
geometry2 = pcd
vis.add_geometry(geometry1)
vis.add_geometry(geometry2)
'''
o3d.visualization.draw_geometries([pcd],
                                  width=800, height=600,
                                  zoom=1,
                                  front=[2, 0.5, -5],
                                  lookat=[0, 0, 0],
                                  up=[-0.0694, -0.9768, 0.2024])
'''
R=line_set.get_rotation_matrix_from_xyz((math.radians(1),0,0))
for i in range(0,360):
    
    line_set.rotate(R, (0.5,0.5,0.5))
    vis.update_geometry(geometry1)
    vis.update_geometry(geometry2)
    print(np.asarray(line_set.points))
    sleep(0.05)
    vis.poll_events()
    vis.update_renderer()
    #o3d.visualization.draw_geometries()
vis.destroy_window()

Let's draw a box using o3d.geometry.LineSet.
[[0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]
 [0. 0. 1.]
 [1. 0. 1.]
 [0. 1. 1.]
 [1. 1. 1.]]
[[ 0.          0.00880236 -0.00865005]
 [ 1.          0.00880236 -0.00865005]
 [ 0.          1.00865005  0.00880236]
 [ 1.          1.00865005  0.00880236]
 [ 0.         -0.00865005  0.99119764]
 [ 1.         -0.00865005  0.99119764]
 [ 0.          0.99119764  1.00865005]
 [ 1.          0.99119764  1.00865005]]
[[ 0.          0.01775433 -0.01714516]
 [ 1.          0.01775433 -0.01714516]
 [ 0.          1.01714516  0.01775433]
 [ 1.          1.01714516  0.01775433]
 [ 0.         -0.01714516  0.98224567]
 [ 1.         -0.01714516  0.98224567]
 [ 0.          0.98224567  1.01714516]
 [ 1.          0.98224567  1.01714516]]
[[ 0.          0.02685321 -0.02548275]
 [ 1.          0.02685321 -0.02548275]
 [ 0.          1.02548275  0.02685321]
 [ 1.          1.02548275  0.02685321]
 [ 0.         -0.02548275  0.97314679]
 [ 1.         -0.02548275  0.97314679

 [ 1.          0.69490504  1.17971466]]
[[ 0.         0.3169873 -0.1830127]
 [ 1.         0.3169873 -0.1830127]
 [ 0.         1.1830127  0.3169873]
 [ 1.         1.1830127  0.3169873]
 [ 0.        -0.1830127  0.6830127]
 [ 1.        -0.1830127  0.6830127]
 [ 0.         0.6830127  1.1830127]
 [ 1.         0.6830127  1.1830127]]
[[ 0.          0.32893539 -0.18610269]
 [ 1.          0.32893539 -0.18610269]
 [ 0.          1.18610269  0.32893539]
 [ 1.          1.18610269  0.32893539]
 [ 0.         -0.18610269  0.67106461]
 [ 1.         -0.18610269  0.67106461]
 [ 0.          0.67106461  1.18610269]
 [ 1.          0.67106461  1.18610269]]
[[ 0.          0.34093558 -0.18898368]
 [ 1.          0.34093558 -0.18898368]
 [ 0.          1.18898368  0.34093558]
 [ 1.          1.18898368  0.34093558]
 [ 0.         -0.18898368  0.65906442]
 [ 1.         -0.18898368  0.65906442]
 [ 0.          0.65906442  1.18898368]
 [ 1.          0.65906442  1.18898368]]
[[ 0.          0.35298423 -0.1916548 ]
 [ 1. 

[[ 0.          0.67106461 -0.18610269]
 [ 1.          0.67106461 -0.18610269]
 [ 0.          1.18610269  0.67106461]
 [ 1.          1.18610269  0.67106461]
 [ 0.         -0.18610269  0.32893539]
 [ 1.         -0.18610269  0.32893539]
 [ 0.          0.32893539  1.18610269]
 [ 1.          0.32893539  1.18610269]]
[[ 0.         0.6830127 -0.1830127]
 [ 1.         0.6830127 -0.1830127]
 [ 0.         1.1830127  0.6830127]
 [ 1.         1.1830127  0.6830127]
 [ 0.        -0.1830127  0.3169873]
 [ 1.        -0.1830127  0.3169873]
 [ 0.         0.3169873  1.1830127]
 [ 1.         0.3169873  1.1830127]]
[[ 0.          0.69490504 -0.17971466]
 [ 1.          0.69490504 -0.17971466]
 [ 0.          1.17971466  0.69490504]
 [ 1.          1.17971466  0.69490504]
 [ 0.         -0.17971466  0.30509496]
 [ 1.         -0.17971466  0.30509496]
 [ 0.          0.30509496  1.17971466]
 [ 1.          0.30509496  1.17971466]]
[[ 0.          0.70673802 -0.17620958]
 [ 1.          0.70673802 -0.17620958]
 [ 0.  

 [ 1.          0.02685321  1.02548275]]
[[ 0.          0.98224567 -0.01714516]
 [ 1.          0.98224567 -0.01714516]
 [ 0.          1.01714516  0.98224567]
 [ 1.          1.01714516  0.98224567]
 [ 0.         -0.01714516  0.01775433]
 [ 1.         -0.01714516  0.01775433]
 [ 0.          0.01775433  1.01714516]
 [ 1.          0.01775433  1.01714516]]
[[ 0.          0.99119764 -0.00865005]
 [ 1.          0.99119764 -0.00865005]
 [ 0.          1.00865005  0.99119764]
 [ 1.          1.00865005  0.99119764]
 [ 0.         -0.00865005  0.00880236]
 [ 1.         -0.00865005  0.00880236]
 [ 0.          0.00880236  1.00865005]
 [ 1.          0.00880236  1.00865005]]
[[ 0.00000000e+00  1.00000000e+00 -1.33226763e-15]
 [ 1.00000000e+00  1.00000000e+00 -1.33226763e-15]
 [ 0.00000000e+00  1.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00 -1.11022302e-15 -1.88737914e-15]
 [ 1.00000000e+00 -1.11022302e-15 -1.88737914e-15]
 [ 0.00000000e+00 -2.1094237

[[ 0.          1.16446302  0.25815524]
 [ 1.          1.16446302  0.25815524]
 [ 0.          0.74184476  1.16446302]
 [ 1.          0.74184476  1.16446302]
 [ 0.          0.25815524 -0.16446302]
 [ 1.          0.25815524 -0.16446302]
 [ 0.         -0.16446302  0.74184476]
 [ 1.         -0.16446302  0.74184476]]
[[ 0.          1.1685826   0.26978855]
 [ 1.          1.1685826   0.26978855]
 [ 0.          0.73021145  1.1685826 ]
 [ 1.          0.73021145  1.1685826 ]
 [ 0.          0.26978855 -0.1685826 ]
 [ 1.          0.26978855 -0.1685826 ]
 [ 0.         -0.1685826   0.73021145]
 [ 1.         -0.1685826   0.73021145]]
[[ 0.          1.17249851  0.28149199]
 [ 1.          1.17249851  0.28149199]
 [ 0.          0.71850801  1.17249851]
 [ 1.          0.71850801  1.17249851]
 [ 0.          0.28149199 -0.17249851]
 [ 1.          0.28149199 -0.17249851]
 [ 0.         -0.17249851  0.71850801]
 [ 1.         -0.17249851  0.71850801]]
[[ 0.          1.17620958  0.29326198]
 [ 1.          1.17620

...

[[ 0.          0.42608721  1.20323318]
 [ 1.          0.42608721  1.20323318]
 [ 0.         -0.20323318  0.42608721]
 [ 1.         -0.20323318  0.42608721]
 [ 0.          1.20323318  0.57391279]
 [ 1.          1.20323318  0.57391279]
 [ 0.          0.57391279 -0.20323318]
 [ 1.          0.57391279 -0.20323318]]
[[ 0.          0.41382536  1.20183611]
 [ 1.          0.41382536  1.20183611]
 [ 0.         -0.20183611  0.41382536]
 [ 1.         -0.20183611  0.41382536]
 [ 0.          1.20183611  0.58617464]
 [ 1.          1.20183611  0.58617464]
 [ 0.          0.58617464 -0.20183611]
 [ 1.          0.58617464 -0.20183611]]
[[ 0.          0.40158976  1.20022527]
 [ 1.          0.40158976  1.20022527]
 [ 0.         -0.20022527  0.40158976]
 [ 1.         -0.20022527  0.40158976]
 [ 0.          1.20022527  0.59841024]
 [ 1.          1.20022527  0.59841024]
 [ 0.          0.59841024 -0.20022527]
 [ 1.          0.59841024 -0.20022527]]
[[ 0.          0.38938413  1.19840112]
 [ 1.          0.38938

[[ 0.          0.09442021  1.07922797]
 [ 1.          0.09442021  1.07922797]
 [ 0.         -0.07922797  0.09442021]
 [ 1.         -0.07922797  0.09442021]
 [ 0.          1.07922797  0.90557979]
 [ 1.          1.07922797  0.90557979]
 [ 0.          0.90557979 -0.07922797]
 [ 1.          0.90557979 -0.07922797]]
[[ 0.          0.08437306  1.0720614 ]
 [ 1.          0.08437306  1.0720614 ]
 [ 0.         -0.0720614   0.08437306]
 [ 1.         -0.0720614   0.08437306]
 [ 0.          1.0720614   0.91562694]
 [ 1.          1.0720614   0.91562694]
 [ 0.          0.91562694 -0.0720614 ]
 [ 1.          0.91562694 -0.0720614 ]]
[[ 0.          0.07445252  1.06472058]
 [ 1.          0.07445252  1.06472058]
 [ 0.         -0.06472058  0.07445252]
 [ 1.         -0.06472058  0.07445252]
 [ 0.          1.06472058  0.92554748]
 [ 1.          1.06472058  0.92554748]
 [ 0.          0.92554748 -0.06472058]
 [ 1.          0.92554748 -0.06472058]]
[[ 0.          0.0646616   1.05720775]
 [ 1.          0.06466

[[ 0.         -0.12433823  0.83196653]
 [ 1.         -0.12433823  0.83196653]
 [ 0.          0.16803347 -0.12433823]
 [ 1.          0.16803347 -0.12433823]
 [ 0.          0.83196653  1.12433823]
 [ 1.          0.83196653  1.12433823]
 [ 0.          1.12433823  0.16803347]
 [ 1.          1.12433823  0.16803347]]
[[ 0.         -0.13003676  0.82101976]
 [ 1.         -0.13003676  0.82101976]
 [ 0.          0.17898024 -0.13003676]
 [ 1.          0.17898024 -0.13003676]
 [ 0.          0.82101976  1.13003676]
 [ 1.          0.82101976  1.13003676]
 [ 0.          1.13003676  0.17898024]
 [ 1.          1.13003676  0.17898024]]
[[ 0.         -0.13554337  0.80997521]
 [ 1.         -0.13554337  0.80997521]
 [ 0.          0.19002479 -0.13554337]
 [ 1.          0.19002479 -0.13554337]
 [ 0.          0.80997521  1.13554337]
 [ 1.          0.80997521  1.13554337]
 [ 0.          1.13554337  0.19002479]
 [ 1.          1.13554337  0.19002479]]
[[ 0.         -0.14085638  0.79883624]
 [ 1.         -0.14085

[[ 0.         -0.20699909  0.51234071]
 [ 1.         -0.20699909  0.51234071]
 [ 0.          0.48765929 -0.20699909]
 [ 1.          0.48765929 -0.20699909]
 [ 0.          0.51234071  1.20699909]
 [ 1.          0.51234071  1.20699909]
 [ 0.          1.20699909  0.48765929]
 [ 1.          1.20699909  0.48765929]]
[[ 0.         -0.20710678  0.5       ]
 [ 1.         -0.20710678  0.5       ]
 [ 0.          0.5        -0.20710678]
 [ 1.          0.5        -0.20710678]
 [ 0.          0.5         1.20710678]
 [ 1.          0.5         1.20710678]
 [ 0.          1.20710678  0.5       ]
 [ 1.          1.20710678  0.5       ]]
[[ 0.         -0.20699909  0.48765929]
 [ 1.         -0.20699909  0.48765929]
 [ 0.          0.51234071 -0.20699909]
 [ 1.          0.51234071 -0.20699909]
 [ 0.          0.48765929  1.20699909]
 [ 1.          0.48765929  1.20699909]
 [ 0.          1.20699909  0.51234071]
 [ 1.          1.20699909  0.51234071]]
[[ 0.         -0.20667603  0.47532233]
 [ 1.         -0.20667

 [ 1.          1.12433823  0.83196653]]
[[ 0.         -0.11844953  0.15718783]
 [ 1.         -0.11844953  0.15718783]
 [ 0.          0.84281217 -0.11844953]
 [ 1.          0.84281217 -0.11844953]
 [ 0.          0.15718783  1.11844953]
 [ 1.          0.15718783  1.11844953]
 [ 0.          1.11844953  0.84281217]
 [ 1.          1.11844953  0.84281217]]
[[ 0.         -0.11237244  0.14644661]
 [ 1.         -0.11237244  0.14644661]
 [ 0.          0.85355339 -0.11237244]
 [ 1.          0.85355339 -0.11237244]
 [ 0.          0.14644661  1.11237244]
 [ 1.          0.14644661  1.11237244]
 [ 0.          1.11237244  0.85355339]
 [ 1.          1.11237244  0.85355339]]
[[ 0.         -0.10610881  0.13581308]
 [ 1.         -0.10610881  0.13581308]
 [ 0.          0.86418692 -0.10610881]
 [ 1.          0.86418692 -0.10610881]
 [ 0.          0.13581308  1.10610881]
 [ 1.          0.13581308  1.10610881]
 [ 0.          1.10610881  0.86418692]
 [ 1.          1.10610881  0.86418692]]
[[ 0.         -0.0996